# Web Scrapping

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import PyPDF2 

In [2]:
def get_job_links(url_search,url_base):
    """ 
    Return a list of links to jobs returned from the url_search link
    """
    # use requests to get the search url contents, convert to a soup object
    r = requests.get(url_search)
    soup = BeautifulSoup(r.content)
    
    # extract the links from the job titles
    jobs = soup.find_all("a",{"class":"SerpJob-titleLink"})
    
    # return a list of job-links
    job_links = []
    for job in jobs:
        job_links.append(f"{url_base}{job.get('href')}")
    return job_links

In [468]:
def get_job_details(urls):
    """
    """
    print("Scrapping RSS Job Search Feed")
    print("="*100)
    job_detail = []
    for url in urls:
        try:
            # get url content, convert to soup
            r = requests.get(url)
            soup = BeautifulSoup(r.content)

            # extract content
            title = soup.find("div",{'class':'ViewJobHeaderTitle'}).text
            company = soup.find("div",{'class':'ViewJobHeaderCompany'}).text
            location = soup.find("span",{'class':'ViewJobHeaderPropertiesLocation'}).text
            desc = soup.find("div",{'class':'viewjob-description'}).text

            # extract key words (skills, education, benefits)
            keys = soup.find_all('li',{'class':'nav-item'})
            key_words = [w.text for w in keys]

            # build result-set
            job_detail.append(
                {'title':title,'company':company,'location':location,'desc':desc,'keywords':key_words}
            )
            # print progress
            print(f"  =>{title[:35]:{40}}| {company[:25]:{25}}| {location} | {key_words[:3]}...")
        except Exception as e:
            print("   =>Skipping Job...")
            continue
    print("="*100)
    return job_detail

In [469]:

url_search = 'https://www.workopolis.com/jobsearch/find-jobs?l=Toronto%2C%20ON&ak=data%20science%20-intern&t=7&sr=10&s=r&job=zD_1ItN-5JXVPUCwXgZvg_joKB7qyQNp5ttncJBtECQ1FoqVIwMR8znJuj3fF7lK'
url_base  ='https://www.workopolis.com'

job_links = get_job_links(url_search, url_base)
job_details = get_job_details(job_links)


Scrapping RSS Job Search Feed
  =>Data Scientist                          | Wish                     | Toronto, ON | ["Master's degree", "Bachelor's", 'MongoDB']...
  =>Data Science Co-op (R&D) - Summer t     | Swift Medical            | Toronto, ON | ["Master's degree", "Bachelor's", 'Machine learning']...
  =>Data Analyst - Operations               | Flair Airlines Ltd.      | Toronto, ON | ["Bachelor's", 'Power BI', 'Microsoft Excel']...
  =>Scene+ Data Scientist                   | Cineplex                 | Toronto, ON | ['Post-secondary education', 'TensorFlow', 'Cloud infrastructure']...
  =>Data Science Manager - Contract         | Scotiabank               | Toronto, ON | ["Master's degree", 'Microsoft Powerpoint', 'Microsoft Word']...
  =>Senior Data Scientist, Trust & Safe     | Thumbtack                | Toronto, ON | ["Master's degree", "Bachelor's", 'Salesforce']...
  =>Data Scientist - Climate/Weather        | RiskThinking.AI          | Toronto, ON | ['Doctoral degree', "

# NLP

In [139]:
# Sumy
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
import nltk # required for sumy
nltk.download('punkt') # required for sumy

# spacy
import spacy
from spacy import displacy
NLP = spacy.load('en_core_web_lg')
from spacy.matcher import PhraseMatcher
# from spacy.matcher import Matcher

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


from tabulate import tabulate
from collections import Counter
import docx2txt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

[nltk_data] Downloading package punkt to /home/ken/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Resume Scrapping

In [6]:
def get_resume_text(path,print_text=True):
    """ 
    Read resume text content from file (pdf or docx)
    Params:
        path: string- path to the resume file
        print_text: boolean.  Prints the text
    Returns: A text object with the resume file content
    """
    
    # open file and get file extension
    file = open(path,'rb')
    file_name, file_ext = os.path.splitext(path)
    
    # extract text content
    if file_ext=='.pdf':
        # read pdf
        pdf = PyPDF2.PdfFileReader(file)
        
        text = """"""
        for page_num in range(pdf.numPages):
            page = pdf.getPage(page_num)
            content = page.extract_text()
            text += content
    elif file_ext=='.docx':
        text = docx2txt.process(file)
    
    if print_text:
        print(text)

    file.close()
    return text

In [8]:
def show_pos(text):    
    """Extracts POS from a spacy document and prints the results"""
    d=[]
    for token in doc:
        d.append({
            "Token":token.text,
            "POS":token.pos_,
            "POS-Desc":spacy.explain(token.pos_),
            "Lemma":token.lemma_,
            "Stop-Word":token.is_stop,
            "POS-Detail":token.tag_,
            "POS-Detail-Desc":spacy.explain(token.tag_)
            }
        )
    print(tabulate(d,headers="keys",tablefmt="github"))

In [9]:
def show_entities(doc,display=True):
    """Print a list of named entities and their descriptions"""
    d=[]
    if doc.ents:
        for entity in doc.ents:
            d.append({
                "Word":entity,
                "Entity":entity.label_,
                "Start":entity.start,
                "Stop":entity.end,
                "Entity-Desc":spacy.explain(entity.label_),
                }
            )
        if display:
            print(tabulate(d,headers="keys",tablefmt="github"))
            print("\n")
            displacy.render(doc,style='ent',jupyter=True)
            print("\n")
    else:
        print("no entities found")
    return d   

In [10]:
def attr_counts(doc,att):
    """Return a table of counts by attributes"""
    if att.lower() == 'pos':
        cnts = doc.count_by(spacy.attrs.POS)
    elif att.lower() =='tag':
        cnts = doc.count_by(spacy.attrs.TAG)
    elif att.lower() =='lemma':
        cnts = doc.count_by(spacy.attrs.LEMMA)
    elif att.lower() =='entity':
        cnts = doc.count_by(spacy.attrs.ENT_TYPE)
    elif att.lower() =='email':
        cnts = doc.count_by(spacy.attrs.LIKE_EMAIL)
    elif att.lower() =='url':
        cnts = doc.count_by(spacy.attrs.LIKE_URL)
    elif att.lower() =='currency':
        cnts = doc.count_by(spacy.attrs.IS_CURRENCY)
    elif att.lower() =='numbers':
        cnts = doc.count_by(spacy.attrs.IS_DIGIT)
        
    # lookup attribute name
    d = {doc.vocab[k].text:v for k,v in cnts.items()}
    return d

### Generate Text Summary

In [502]:
def get_text_summary(text,text_source,summarizer,num_sentences,print_summary=True):
    """
    Generate a text summary from a document (extractive)
    Params:
        text: plain text or path to a document
        text_source: text or file
        summarizer: text-summarization method. lex-rank, text-rank,lsa
        num_sentences: Number of summary sentences to generate
        print_summary: T/F
    Returns:
        A list of sentences.
    """
    # Create parser from text-source
    if text_source=='text':
        parser = PlaintextParser.from_string(text,Tokenizer('english'))
    elif text_source=='file':
        parser = PlaintextParser.from_file(text,Tokenizer('english'))
    else:
        raise Exception(f"text source must be 'text' or 'file'!")
        
    # Select summarizer method
    if summarizer=='lex-rank':
        # Lex Rank:  A graph-based summarization method that uses keyword extractions
        s = LexRankSummarizer()
    elif summarizer=='text-rank':
        # Text rank: A graph-based summarization method that uses keyword extractions
        s = TextRankSummarizer()
    elif summarizer=='lsa':
        # Latent semantic analysis 
        # Combines term frequency with singular value decomposition
        s = LsaSummarizer(Stemmer("english"))
        s.stop_words = get_stop_words("english")
    else:
        raise Exception(f"Summarizer must be 'lex-rank','luhn','text-rank' or 'lsa'!")
    
    # create summary
    summary = s(parser.document,num_sentences)
    
    # print summary
    text = ""
    if print_summary:
        print(f'Text Summary:Top {num_sentences} sentences using a {summarizer} summarizer')
        print('='*100)
        for sentence in summary:
            text += " "+ str(sentence)
            print(sentence)
        print('='*100,'\n')
    return text

### Keyword Analysis

In [12]:
def get_terms(path):
    """
    Read a csv of data science terms & return themn as a list
    Params: path -string
    Returns: A list of terms
    """
    with open(path,'r') as f:
        lst = [line.lower().strip() for line in f.readlines()]
    return set(lst)

In [ ]:
def extract_keywords(text,num, print_words=True):
    """
    Extracts keywords from a text object.  Ignores common job positng phrases
    Params:
        text: string text object
        num: integer.  The number of keywords to extract
        print_words: boolean. Prints the output
    Returns: A list of the top keywords in the text object
    """
    words = []
    pos = ['NOUN','PROPN']
    common_job_keys = ['experience','job','ability','posting','process','date','stakeholders','skills','solutions','project']
    tokens = NLP(text.lower())
    
    for token in tokens:
        if (token.text not in NLP.Defaults.stop_words and token.text.lower() not in common_job_keys and token.pos_ in pos):
            words.append(token.text)
            
    # get the top keywords by frequency
    keyword_frequency = [word for word in Counter(words).most_common(num)]
    keywords = [word[0] for word in keyword_frequency]
    
    if print_words:
        print(keyword_frequency)
    return keywords            

In [467]:
# extract keywords from the job descriptions
keys = []
for job in job_details:
    keys.extend(extract_keywords(job['desc'],50,False))
keys = set(keys)

In [466]:
# extract keywords from the job keywords
keys = []
for job in job_details:
    keys.extend(job['keywords'])
keys = set(keys)

In [77]:
def get_phrases_from_keywords(keywords):
    lst = []
    for keyword in keywords:
        if " " in keyword:
            lst.append(keyword.lower().replace(" ","-"))
        lst.append(keyword.lower())
    return set(lst)

In [206]:
def get_phrase_matches(keywords,text):
    """
    spacy 3.5 has a built in fuzzy matching option
    """
    # when spacy 3.5 is available on anaconda
#     matcher = Matcher(NLP.vocab)
#     patterns = [{"TEXT": {"FUZZY": "modelling"}},{"TEXT": {"FUZZY": "machine learning"}}]

    # get the term list
    terms = get_phrases_from_keywords(keywords)
    
    # convert text to tokens
    doc = NLP(text.lower())

    # phrase matching
    patterns = [NLP(text) for text in terms]

    # add to matcher
    matcher = PhraseMatcher(NLP.vocab)
    matcher.add('KEYWORDS',patterns)

    # search for phrase matches
    found_matches = matcher(doc)

    matched_phrases = []
    for match_id, start, end in found_matches:
        matched_phrases.append(doc[start:end].text)

    return dict(Counter(matched_phrases)) 

In [388]:
def compare_keywords(resume,job,keyword_path):
    """
    """
    # path to csv of terms
    keys = get_phrases_from_keywords(get_terms(keyword_path))
    
    # match phrases
    matches_resume = get_phrase_matches(keys,resume)
    matches_job = get_phrase_matches(keys,job)

    # convert to dataframes
    df_resume = pd.DataFrame.from_dict(matches_resume,orient='index',columns=['resume'])
    df_job = pd.DataFrame.from_dict(matches_job,orient='index',columns=['job'])
    
    # combine results & sort by job keywords
    df = df_job.join(df_resume)
    df.sort_values(by=['job'], ascending=False,inplace=True)
    
    # calculate the weighted difference in keyword frequencies
    df['wt'] = df['job']/df['job'].sum()
    df['wt-score'] = df['wt']*(df['resume']-df['job'])
    score = df['wt-score'].sum()
    
    # calculate keyword coverage ratio
    matches = df.query('~resume.isnull()').shape[0]
    total = df.shape[0]
    coverage = matches/total
    
    # get missing
    missing = set(df.query('resume.isnull()').index)
    
    
    return df, score, coverage, missing

In [245]:
def compare_job_specific_keywords(resume, job_keywords):
    """
    """
    # match phrases
    matches = get_phrase_matches(job_keywords,resume)
    
    # add missing job keywords to the dict
    for key in job_keywords:
        if key.lower() in matches.keys():
            continue
        else:
            matches[key.lower()]=0
    
    # convert to dateframe
    df = pd.DataFrame.from_dict(matches,orient='index',columns=['resume'])
    
    return df

In [454]:
def plot_keyword_comparison(resume, job_details, keyword_path):
    """ 
    Plot Job keywords vs Resume keywords
    Params: df- dataframe output from compare_keywords
    Returns: None
    """
    
    # compare common keywords, split into matches/no-matches
    df, score, coverage, missing = compare_keywords(resume,job_details['desc'],keyword_path)
    df_missing  = df.query('resume.isnull()')
    df_matching = df.query('~resume.isnull()')
    
    title = f"Keyword Coverage={round(coverage*100,2)}% | Weighted Average={round(score,2)}"
    
    # compare to job specific keywords
    df_spec = compare_job_specific_keywords(resume, job_details['keywords'])
    
    col_scale = [
        [0,'rgba(252,76,1,0.9)'],
        [0.25,'rgba(252,76,1,0.5)'],
        [0.5,'rgba(83,128,141,0.25)'],
        [0.75,'rgba(83,128,141,0.5)'],
        [1.0,'rgba(83,128,141,0.9)']
    ]
    
    # create plots
    fig = make_subplots(
        rows=1,
        cols=3,
        subplot_titles = ('Matching Keywords','Missing Keywords','Job Specific Keywords'),
        column_widths = [0.5,0.25,0.25]
    )
    # matching keywords
    fig.add_trace(
        go.Bar(
            name='Weighted Frequency',
            y = df_matching.index,
            x = df_matching['wt-score'],
            orientation='h',
            marker = dict(color=df_matching['wt-score'],colorscale=col_scale)
        ),row=1,col=1
    )
    # missing keywords
    fig.add_trace(
        go.Bar(
            name='Missing Keywords',
            y = df_missing.job,
            x = df_missing.index,
            marker = dict(color='rgba(252,76,1,0.8)')
        ),row=1,col=2
    )
    # job specific keywords
    fig.add_trace(
        go.Bar(
            name='Job Specific Keywords',
            y = df_spec.resume,
            x = df_spec.index,
            marker = dict(color='rgba(252,76,1,0.8)')
        ),row=1,col=3
    )
    
    # adjusments
    fig.update_xaxes(tickangle=90,row=1,col=2)
    fig.update_xaxes(tickangle=90,row=1,col=3)
    fig.update_xaxes(range=[-2,2],row=1,col=1)
    fig.update_xaxes(showgrid=False)
    fig.update_layout(
        template='plotly_white',
        title=title,
        width = 900,
        height=500,
        showlegend=False
    )
    fig.show()

## Calculate Similarities

In [26]:
def preprocess_text(text):
    """
    Preprocess text documents in preparation for analysis
    Params:
        text: a text object
    Returns: 
        A text object after converting to lowercase lemmas, removing stop-words,punctuation, numbers, and emails
    """
    
    # replace carriage returns and convert to lowercase
    doc = text.replace("\n","").replace("  ","").lower()
    
    # tokenize
    tokens = NLP(doc)

    text_list = []
    for token in tokens:
        if token.text in NLP.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.is_digit:
            continue
        if token.like_email:
            continue
        if token.lemma_=='-PRON-':
            continue
            
        # lemmetize
        text_list.append(token.lemma_)
    return " ".join(text_list)

In [27]:
def calc_similarity(doc1,doc2):
    """ 
    Calculate the cosine similarity between two documents
    Params:
        doc1 - text document 
        doc2 - text document
    Returns: The cosine similairty score between the documents
    """
    d1 = preprocess_text(doc1)
    d2 = preprocess_text(doc2)
    
    X = TfidfVectorizer().fit_transform([d1,d2])
    score = cosine_similarity(X[0],X[1])
    return score[0][0]

In [390]:
def calc_similarities(resume, jobs,keyword_path):
    """ 
    Calculate the cosine similarity between a resume and a list of job descriptions
    Params:
        resume: a resume (text)
        jobs: a dictionary of jobs listings (output from get_job_details)
    Returns: A dataframe of job title, company, location and similarity score
    """
    scores = []
    titles = []
    companies =[]
    locs = []
    key_cov = []
    key_wt = []
    key_missing = []
    for job in jobs:
        titles.append(job['title'])
        companies.append(job['company'])
        locs.append(job['location'])
        scores.append(calc_similarity(resume, job['desc']))
        _,wt,cov,missing = compare_keywords(resume,job['desc'],keyword_path)
        key_cov.append(cov)
        key_wt.append(wt)
        key_missing.append(missing)
    
    data = list(zip(titles,companies,locs, scores, key_cov,key_wt,key_missing ))
    df = pd.DataFrame(data,columns = ['title','company','location','similarity-score','keyword-coverage','keyword-wt','keywords-missing'])
    
    return df.sort_values(by=['similarity-score'], ascending=False)

In [458]:
def plot_similarity_scores(df_similarities):
    """
    """
    
    # min-max scale the keyword weights
    df = df_similarities.copy()
    df['keyword-weight'] = (df['keyword-wt']-min(df['keyword-wt']))/(max(df['keyword-wt'])-min(df['keyword-wt']))
    
    # custom colorscale
    col_scale = [
        [0,'rgba(252,76,1,1.0)'],
        [0.25,'rgba(252,76,1,0.5)'],
        [0.5,'rgba(83,128,141,0.25)'],
        [0.75,'rgba(83,128,141,0.5)'],
        [1.0,'rgba(83,128,141,1.0)']
    ]
        
    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(
            x = df['keyword-coverage'],
            y = df['similarity-score'],
            mode='markers',
            text = df['title'],
            marker = dict(
                size=df['keyword-weight']*75,
                color = df['similarity-score'],
#                 colorscale='Cividis_r',
                colorscale=col_scale
            )
        )
    )
    fig.update_xaxes(title='Keyword Coverge (%)')
    fig.update_yaxes(title='Similarity Score')
    fig.update_layout(
        template='plotly_white',
        title='Resume Similarity to Job Listings',
        width=800,
        height=500
    )
    fig.show()    

In [470]:
# get resume
r = get_resume_text('data/resume.docx',False)
# cl = get_resume_text('data/cover-letter.docx',True)

# calc similarities with each job
df_r = calc_similarities(r,job_details,'data/terms.csv')
df_r

,title,company,location,similarity-score,keyword-coverage,keyword-wt,keywords-missing
8,Sr. IT Data Analyst,TD Bank,"Toronto, ON",0.335239,0.680000,5.122807,"{hadoop, metadata, banking, linux, infrastruct..."
13,Sr. Data Analyst (Pharma),Tiger Analytics,"Toronto, ON",0.267301,0.708333,3.714286,"{communication skills, metadata, ai, pharma, a..."
14,"Manager, Machine Learning and Advanced Analytics",MLSE,"Toronto, ON",0.259009,0.657895,3.298507,"{mathematics, regression, communication skills..."
4,Data Science Manager - Contract,Scotiabank,"Toronto, ON",0.258833,0.702703,2.075472,"{regression, banking, computer science, estima..."
0,Data Scientist,Wish,"Toronto, ON",0.249669,0.833333,5.033333,"{mongodb, master’s degree, insights}"
9,Data Scientist,Arup,"Toronto, ON",0.239995,0.682927,-0.263889,"{random forest, mathematics, ai, ml, insurance..."
18,"Senior Manager, Data and AI Risk",Scotiabank,"Toronto, ON",0.228092,0.600000,3.250000,"{ai, metadata, banking, financial services, ma..."
12,Senior Data Analyst,Sun Life,"Toronto, ON",0.221648,0.678571,2.760000,"{banking, computer science, financial services..."
2,Data Analyst - Operations,Flair Airlines Ltd.,"Toronto, ON",0.221136,0.807692,5.621622,"{mathematics, communication skills, insurance,..."
6,Data Scientist - Climate/Weather,RiskThinking.AI,"Toronto, ON",0.207322,0.600000,4.895833,"{matlab, ai, computer science, c++, ml, infras..."


In [471]:
# plot similarities to resume
plot_similarity_scores(df_r)

In [472]:
# plot resume comparision to a specific job 
plot_keyword_comparison(r,job_details[14],'data/terms.csv')

In [474]:
# get resume
cl = get_resume_text('data/cover_letter.docx',False)


# calc similarities with each job
df_cl = calc_similarities(cl,job_details,'data/terms.csv')
df_cl

,title,company,location,similarity-score,keyword-coverage,keyword-wt,keywords-missing
9,Data Scientist,Arup,"Toronto, ON",0.377605,0.268293,-2.208333,"{python, data mining, pipelines, ml, insights,..."
13,Sr. Data Analyst (Pharma),Tiger Analytics,"Toronto, ON",0.361386,0.291667,-1.904762,"{consulting, project management, metadata, ai,..."
0,Data Scientist,Wish,"Toronto, ON",0.328269,0.388889,0.033333,"{data visualization, engineering, python, pipe..."
3,Scene+ Data Scientist,Cineplex,"Toronto, ON",0.316151,0.297297,-2.030303,"{snowflake, python, infrastructure, kubernetes..."
8,Sr. IT Data Analyst,TD Bank,"Toronto, ON",0.310040,0.160000,-0.596491,"{data profiling, project management, metadata,..."
1,Data Science Co-op (R&D) - Summer term,Swift Medical,"Toronto, ON",0.309663,0.333333,-2.037736,"{predictive, ai, python, engineering, mentorin..."
12,Senior Data Analyst,Sun Life,"Toronto, ON",0.288357,0.250000,-0.820000,"{python, insights, testing, sas, sql, options,..."
14,"Manager, Machine Learning and Advanced Analytics",MLSE,"Toronto, ON",0.260264,0.263158,-0.820896,"{project management, pipelines, insights, test..."
15,Machine Learning Engineer,Autodesk,"Toronto, ON",0.259784,0.272727,-0.673469,"{data-analytics, python, ml, insights, cluster..."
2,Data Analyst - Operations,Flair Airlines Ltd.,"Toronto, ON",0.255140,0.269231,-0.054054,"{data profiling, python, pipelines, insights, ..."


In [475]:
# plot similarities to cover letter
plot_similarity_scores(df_cl)

In [509]:
# compare job summary and cover letter summary
summary_job = get_text_summary(job_details[14]['desc'],'text','lsa',3)
summary_cl = get_text_summary(cl,'text','lsa',3)

calc_similarity(summary_cl,summary_job)


Text Summary:Top 3 sentences using a lsa summarizer
Company Description At Maple Leaf Sports & Entertainment Partnership (MLSE), we are committed to creating an inclusive workplace that is representative of our community and where all employees feel they belong and can reach their full potential.
Use Microsoft Power BI, Tableau and other Business Intelligence tools to craft and build executive, sales, marketing, and operational dashboards that provide key insights and performance metrics.
Conduct in-depth analyses using traditional and sophisticated methods across various data sources and topic areas, including ticketing, digital and social media, retail, and food & beverage.

Text Summary:Top 3 sentences using a lsa summarizer
I feel that this career opportunity offered with your organization is a strong match for my experience in data science, programming, and quantitative investment analysis.
As a Chartered Financial Analyst with a Master’s degree in Data Science I have over 20+ yea

0.03804131693271844